In [1]:
# kpgpt2 << 고민 
import kogpt2_transformers
# gpt2 모델 사용해보기
from transformers import GPT2ForSequenceClassification
from transformers import AutoTokenizer

c:\Users\KDP-30\anaconda3\envs\WEB_AI\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import torch
import RNNutils2 as utils
device = "cuda" if torch.cuda.is_available() else 'cpu'

In [3]:
dataDF=pd.read_csv('ly4.csv')
dataDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712693 entries, 0 to 712692
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   0       712693 non-null  int64
 1   1       712693 non-null  int64
 2   2       712693 non-null  int64
 3   3       712693 non-null  int64
dtypes: int64(4)
memory usage: 21.7 MB


In [4]:
import numpy as np
target=dataDF[['3']]
feature=dataDF.drop(columns=['3'])

In [5]:
from torch.utils.data import DataLoader

trainDS=utils.MyDataSet(feature,target)
trainDL=DataLoader(trainDS,batch_size=32,pin_memory=True)

In [6]:

import json
with open("token_to_idx.json", "r", encoding="utf-8") as f:
    token_to_idx=json.load(f)
with open("idx_to_token.json", "r", encoding="utf-8") as f:
    idx_to_token=json.load(f)


In [7]:
tokenizer=AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token=tokenizer.eos_token
model= GPT2ForSequenceClassification.from_pretrained(pretrained_model_name_or_path='gpt2',num_labels=len(token_to_idx)).to(device)
model.config.pad_token_id=model.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:

import torch.optim as optim
import torch.nn as nn
from torchmetrics.classification import MulticlassF1Score
import torch.optim.lr_scheduler as lr_scheduler                         

optimizer=optim.Adam(model.parameters(),lr=0.005)
lossfun=nn.CrossEntropyLoss()
scorefun=MulticlassF1Score(num_classes=len(token_to_idx))
scheduler=lr_scheduler.ReduceLROnPlateau(optimizer,patience=10,factor=0.3,mode='max')


In [9]:
EPOCH=1
TV=utils.Train_val(trainDL,model,optimizer,lossfun,scorefun)
train=TV.train(EPOCH,scheduler,5)

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 10316224800 bytes.